# セルオートマトンであることを活かして直近3枚だけを保持する
ということをすればメモリの大幅な削減になる。Pythonで書く。

x+y+z=kの1階層はz=0の面に無駄なく射影できるので、ここで次元が1個落ちる。
だが射影したあとの座標をセルオートマトンに移すのはちょっと難しい。



In [1]:
# インポート系
!pip install pillow
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
from sys import setrecursionlimit
from copy import deepcopy
import time
from functools import lru_cache
import sys
from IPython.utils.py3compat import u_format
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import imageio
import os
import csv
import subprocess

# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 準備
setrecursionlimit(10**8)  # 再帰計算の上限回数を上げる
def LL(h,w,e=0): return [[e for i in range(w)] for j in range(h)]
def LLL(a,b,c,e=0): return [LL(b,c,e) for _ in range(a)]
def LLLL(a,b,c,d,e=0): return [LLL(b,c,d,e) for _ in range(a)]

Mounted at /content/drive


In [2]:
# 変数名定義など

p = 3  # 禁止手数
s = 310  # 探索範囲
ver = 2

csv_path = "/content/drive/MyDrive/result_sbnim.csv"  # csvファイル
png_dir = '/content/drive/MyDrive/研究 組合せ/３次元スライスp={}, s={} ({})/'.format(p,s,ver)  # pngファイルの置き場所
gif_filename_1 = 'output_part1.gif'
gif_filename_2 = 'output_part2.gif'
gif_filename_final = 'p={}.gif'.format(p)

colormap = plt.get_cmap('hsv')  # カラーマップ
width, height = 6400, 4800
margin = 50  # 余白のサイズ

# 画像にステータスを書き込む
font_path = '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf'
font_size = 60
status_position = (2600,100)
font = ImageFont.truetype(font_path, font_size)

# 該当フォルダがない場合、新しくフォルダを作成する
if not os.path.exists(png_dir):
    os.makedirs(png_dir)

In [3]:
# 配列の初期化
f2 = LL(s,s,0)
f3 = [LL(s,s,0) for i in range(4)]
points = []

# 関数の定義
def g2(x,y):
    return int(f2[x][y] <= p)

def g3(t,x,y):
    return int(f3[t][x][y] <= p)

# 描画用
Color = []
M = 8
for i in range(M):
    Color.append(tuple(int(255 * c) for c in colormap(i/M)[:3]))
ps = 1

In [6]:
# 1次元の計算
for i in range(s):
    f2[i][0] = min(i, p+1)
    f2[0][i] = min(i, p+1)

# 2次元の計算
for i in range(1, s):
    for j in range(1, s):
        v = f2[i-1][j] + f2[i][j-1] - f2[i-1][j-1] - 2*g2(i-1, j-1) + g2(i-1, j) + g2(i, j-1)
        f2[i][j] = v

# 3次元の計算 最初の3枚を埋める
# 1
f3[0][0][0] = f2[0][0]  # (0,0,0)

# 2
f3[1][0][0] = f3[1][0][1] = f3[1][1][0] = f2[0][1]  # (1,0,0)

# 3
f3[2][0][0] = f3[2][2][0] = f3[2][0][2] = f2[0][2]  # (2,0,0)
f3[2][1][1] = f3[2][1][0] = f3[2][0][1] = f2[1][1]  # (1,1,0)

S = set([])

# それ以降
for k in range(3, s):
    # 300, 030, 003 / 210, 201. 120, 021, 102, 012 / 111
    # ↓
    # 300, 030, 000 / 210, 200, 120, 020, 100, 010 / 110
    # のように、z座標は自動的に決まるので0に統一してよい

    # 進捗を表示
    if k%50 == 0:
        print(k, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() + 9*3600)))


    # 実質2dの場所を埋める x+y=k
    for x in range(k+1):
        y = k-x
        z = 0
        f3[3][x][y] = f3[3][x][z] = f3[3][z][y] = f2[x][y]

    # それ以外のところを漸化式に従って埋める
    for x in range(1, k):
        for y in range(1, k):
            if x+y>=k: continue  # こう書くのはちょっと効率悪いが一旦これで
            # z = k-x-y
            # print(x,y,z)
            f3[3][x][y] = f3[2][x-1][y] + f3[2][x][y-1] + f3[2][x][y] \
                            - (f3[1][x][y-1] + f3[1][x-1][y] + f3[1][x-1][y-1]) \
                            + f3[0][x-1][y-1] \
                            + g3(2,x,y) + g3(2,x,y-1) + g3(2,x-1,y) \
                            -2 *(g3(1,x,y-1) + g3(1,x-1,y) + g3(1,x-1,y-1)) \
                            +3 * g3(0,x-1,y-1)

            # if not (p-3 <= f3[3][x][y] <= p+4):
            #     print(k,x,y,f3[3][x][y])
            #     sys.exit()
            if k>=p:
                S.add((f3[0][x-1][y-1], tuple(sorted([f3[1][x][y-1], f3[1][x-1][y], f3[1][x-1][y-1]])), \
                        tuple(sorted([f3[2][x-1][y], f3[2][x][y-1], f3[2][x][y]])), f3[3][x][y]))


    # 描画
    #if k%10 == 0 and k >= p:  # 画像をつくるタイミング
    if k == 200:
        Xm, XM, Ym, YM = 10000,0,10000,0
        for x in range(k+1):
            for y in range(k+1):
                if x+y>k: continue
                z = k-x-y
                X = x * (3**.5/6 + 1/2) + y * (3**.5/6 - 1/2) - 3**.5/3 * z
                Y = x * (3**.5/6 - 1/2) + y * (3**.5/6 + 1/2) - 3**.5/3 * z
                Xm = min(Xm, X)
                XM = max(XM, X)
                Ym = min(Ym, Y)
                YM = max(YM, Y)
                points.append((X,Y,f3[3][x][y]))

        image = Image.new("RGB", (width, height), "white")
        draw = ImageDraw.Draw(image)
        Scale = min((width - 2 * margin) / (XM - Xm), (height - 2 * margin) / (YM - Ym))

        # ステータス情報を追記
        status = "p={}, k={}, s={}".format(p,k,s)
        draw.text(status_position, status, fill=(0,0,0), font=font)

        for u,v,i in points:
            plot_x = Scale * (u - Xm) + margin
            plot_y = Scale * (YM - v) + margin
            rgb_color =  Color[i-(p-3)]
            draw.ellipse((plot_x - ps, plot_y - ps, plot_x + ps, plot_y + ps), fill=rgb_color)
        image.save(png_dir + "p={}k={}s={}.png".format(p,"{:03}".format(k),s))

    # 表をずらす
    f3.pop(0)
    f3.append(LL(s,s,0))
    points = []

print(S)

50 2023-12-07 15:51:31
100 2023-12-07 15:51:32
150 2023-12-07 15:51:39
200 2023-12-07 15:51:44
250 2023-12-07 15:51:57
300 2023-12-07 15:52:11
{(4, (4, 5, 5), (4, 4, 6), 4), (6, (5, 5, 6), (3, 6, 6), 6), (4, (3, 4, 5), (4, 5, 5), 4), (5, (3, 5, 6), (4, 5, 6), 4), (4, (3, 4, 4), (3, 5, 5), 5), (6, (5, 6, 6), (5, 5, 5), 4), (3, (5, 5, 5), (4, 4, 6), 5), (4, (3, 4, 6), (5, 5, 5), 4), (5, (4, 4, 5), (2, 5, 5), 5), (5, (4, 6, 6), (5, 5, 5), 4), (5, (3, 6, 6), (6, 6, 6), 6), (3, (5, 6, 6), (5, 5, 6), 5), (5, (4, 4, 5), (3, 5, 5), 6), (6, (4, 6, 6), (4, 4, 5), 3), (4, (3, 4, 5), (5, 5, 5), 5), (4, (3, 3, 6), (6, 6, 6), 6), (5, (3, 5, 6), (5, 5, 6), 5), (4, (4, 4, 6), (3, 5, 5), 4), (4, (4, 5, 5), (4, 4, 5), 3), (3, (5, 5, 5), (4, 4, 5), 4), (4, (4, 4, 5), (3, 5, 5), 5), (4, (3, 4, 5), (3, 5, 5), 4), (4, (5, 5, 6), (5, 5, 5), 3), (3, (5, 6, 6), (4, 5, 5), 3), (6, (5, 5, 5), (4, 5, 6), 6), (5, (3, 5, 6), (4, 5, 5), 3), (4, (4, 4, 4), (3, 4, 6), 6), (5, (5, 5, 5), (3, 6, 6), 6), (5, (4, 4, 6), (

In [7]:
for i in S:
    print(i)

(4, (4, 5, 5), (4, 4, 6), 4)
(6, (5, 5, 6), (3, 6, 6), 6)
(4, (3, 4, 5), (4, 5, 5), 4)
(5, (3, 5, 6), (4, 5, 6), 4)
(4, (3, 4, 4), (3, 5, 5), 5)
(6, (5, 6, 6), (5, 5, 5), 4)
(3, (5, 5, 5), (4, 4, 6), 5)
(4, (3, 4, 6), (5, 5, 5), 4)
(5, (4, 4, 5), (2, 5, 5), 5)
(5, (4, 6, 6), (5, 5, 5), 4)
(5, (3, 6, 6), (6, 6, 6), 6)
(3, (5, 6, 6), (5, 5, 6), 5)
(5, (4, 4, 5), (3, 5, 5), 6)
(6, (4, 6, 6), (4, 4, 5), 3)
(4, (3, 4, 5), (5, 5, 5), 5)
(4, (3, 3, 6), (6, 6, 6), 6)
(5, (3, 5, 6), (5, 5, 6), 5)
(4, (4, 4, 6), (3, 5, 5), 4)
(4, (4, 5, 5), (4, 4, 5), 3)
(3, (5, 5, 5), (4, 4, 5), 4)
(4, (4, 4, 5), (3, 5, 5), 5)
(4, (3, 4, 5), (3, 5, 5), 4)
(4, (5, 5, 6), (5, 5, 5), 3)
(3, (5, 6, 6), (4, 5, 5), 3)
(6, (5, 5, 5), (4, 5, 6), 6)
(5, (3, 5, 6), (4, 5, 5), 3)
(4, (4, 4, 4), (3, 4, 6), 6)
(5, (5, 5, 5), (3, 6, 6), 6)
(5, (4, 4, 6), (3, 4, 6), 5)
(4, (3, 4, 6), (4, 5, 6), 4)
(5, (4, 4, 4), (3, 4, 4), 5)
(3, (4, 5, 6), (4, 5, 5), 5)
(6, (4, 5, 6), (3, 4, 6), 5)
(4, (2, 3, 3), (4, 4, 5), 3)
(5, (4, 6, 6),